### Imports

In [52]:
from typing import Optional, Union, Any
from collections.abc import Iterable
from abc import ABC, abstractmethod
import numbers

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

### Classe NameManager

A classe NameManager provê uma forma conveniente de dar nomes intuitivos para tensores que resultam de operações. A idéia é tornar mais fácil para o usuário das demais classes qual operação gerou qual tensor. Ela provê os seguintes métodos públicos: 

- reset(): reinicia o sistema de gestão de nomes.
- new(<basename>: str): retorna um nome único a partir do nome de base passado como argumento. 
  
Como indicado no exemplo abaixo da classe, a idéia geral é que uma sequência de operações é feita, os nomes dos tensores sejam os nomes das operações seguidos de um número. Se forem feitas 3 operações de soma e uma de multiplicação, seus tensores de saída terão os nomes "add:0", "add:1", "add:2" e "prod:0".

In [53]:

class NameManager:
    _counts = {}

    @staticmethod
    def reset():
        NameManager._counts = {}

    @staticmethod
    def _count(name):
        if name not in NameManager._counts:
            NameManager._counts[name] = 0
        count = NameManager._counts[name]
        return count

    @staticmethod
    def _inc_count(name):
        assert name in NameManager._counts, f'Name {name} is not registered.'
        NameManager._counts[name] += 1

    @staticmethod
    def new(name: str):
        count = NameManager._count(name)
        tensor_name = f"{name}:{count}"
        NameManager._inc_count(name)
        return tensor_name

# exemplo de uso
print(NameManager.new('add'))
print(NameManager.new('in'))
print(NameManager.new('add'))
print(NameManager.new('add'))
print(NameManager.new('in'))
print(NameManager.new('prod'))

NameManager.reset()

add:0
in:0
add:1
add:2
in:1
prod:0


### Classe Tensor

Deve ser criada uma classe `Tensor` representando um array multidimensional.

In [54]:
class Tensor:
    """Classe representando um array multidimensional.

    Atributos:

    - _arr  (privado): dados internos do tensor como
        um array do numpy com 2 dimensões (ver Regras)

    - _parents (privado): lista de tensores que foram
        usados como argumento para a operação que gerou o
        tensor. Será vazia se o tensor foi inicializado com
        valores diretamente. Por exemplo, se o tensor foi
        resultado da operação a + b entre os tensores a e b,
        _parents = [a, b].

    - requires_grad (público): indica se devem ser
        calculados gradientes para o tensor ou não.

    - grad (público): Tensor representando o gradiente.

    """
    
    _tensor_counter = 0  # Contador global para nomes únicos

    def __init__(self,
                 # Dados do tensor. Além dos tipos listados,
                 # arr também pode ser do tipo Tensor.
                 arr: Union[np.ndarray, list, numbers.Number, Any],
                 # Entradas da operacao que gerou o tensor.
                 # Deve ser uma lista de itens do tipo Tensor.
                 parents: list[Any] = None,
                 # se o tensor requer o calculo de gradientes ou nao
                 requires_grad: bool = True,
                 # nome do tensor
                 name: str = '',
                 # referência para um objeto do tipo Operation (ou
                 # subclasse) indicando qual operação gerou este
                 # tensor. Este objeto também possui um método
                 # para calcular a derivada da operação.
                 operation=None):
        """Construtor

        O construtor deve permitir a criacao de tensores das seguintes formas:

            # a partir de escalares
            x = Tensor(3)

            # a partir de listas
            x = Tensor([1,2,3])

            # a partir de arrays
            x = Tensor(np.array([1,2,3]))

            # a partir de outros tensores (construtor de copia)
            x = Tensor(Tensor(np.array([1,2,3])))

        Para isto, as seguintes regras devem ser obedecidas:

        - Se o argumento arr não for um array do numpy,
            ele deve ser convertido em um. Defina o dtype do
            array como float de forma a permitir que NÃO seja
            necessário passar constantes float como Tensor(3.0),
            mas possamos criar um tensor apenas com Tensor(3).

        - O atributo _arr deve ser uma matriz, isto é,
            ter 2 dimensões (ver Regras).

        - Se o argumento arr for um Tensor, ele deve ser
            copiado (cuidado com cópias por referência).

        - Se arr for um array do numpy com 1 dimensão,
            ele deve ser convertido em uma matriz coluna.

        - Se arr for um array do numpy com dimensão maior
            que 2, deve ser lançada uma exceção.

        - Tensores que não foram produzidos como resultado
            de uma operação não têm pais nem operação.
            Os nomes destes tensores devem seguir o formato in:3.
        """
        
        # Inicializar parents como lista vazia se None
        if parents is None:
            parents = []
        
        # Processar o input arr
        if isinstance(arr, Tensor):
            # Construtor de cópia - copia o array interno
            self._arr = arr._arr.copy()
        elif isinstance(arr, np.ndarray):
            # Verificar dimensionalidade
            if arr.ndim > 2:
                raise ValueError(f"Arrays com mais de 2 dimensões não são suportados. Recebido: {arr.ndim} dimensões")
            elif arr.ndim == 0:
                # Escalar numpy -> matriz 1x1
                self._arr = np.array([[arr.item()]], dtype=float)
            elif arr.ndim == 1:
                # Vetor -> matriz coluna
                self._arr = arr.reshape(-1, 1).astype(float)
            else:
                # arr.ndim == 2, já é uma matriz
                self._arr = arr.astype(float)
        elif isinstance(arr, (list, tuple)):
            # Converter lista/tupla para numpy array
            np_arr = np.array(arr, dtype=float)
            if np_arr.ndim > 2:
                raise ValueError(f"Listas com mais de 2 dimensões não são suportadas. Recebido: {np_arr.ndim} dimensões")
            elif np_arr.ndim == 0:
                # Escalar -> matriz 1x1
                self._arr = np.array([[np_arr.item()]], dtype=float)
            elif np_arr.ndim == 1:
                # Vetor -> matriz coluna
                self._arr = np_arr.reshape(-1, 1)
            else:
                # np_arr.ndim == 2, já é uma matriz
                self._arr = np_arr
        elif isinstance(arr, numbers.Number):
            # Escalar -> matriz 1x1
            self._arr = np.array([[float(arr)]], dtype=float)
        else:
            # Tentar converter para numpy array
            try:
                np_arr = np.array(arr, dtype=float)
                if np_arr.ndim > 2:
                    raise ValueError(f"Dados com mais de 2 dimensões não são suportados. Recebido: {np_arr.ndim} dimensões")
                elif np_arr.ndim == 0:
                    self._arr = np.array([[np_arr.item()]], dtype=float)
                elif np_arr.ndim == 1:
                    self._arr = np_arr.reshape(-1, 1)
                else:
                    self._arr = np_arr
            except:
                raise TypeError(f"Tipo não suportado para criação de Tensor: {type(arr)}")
        
        # Garantir que _arr sempre tenha 2 dimensões
        if self._arr.ndim != 2:
            raise RuntimeError(f"Erro interno: _arr deve ter 2 dimensões, mas tem {self._arr.ndim}")
        
        # Configurar atributos
        self._parents = parents
        self.requires_grad = requires_grad
        self.grad = None
        self._operation = operation
        
        # Configurar nome
        if name:
            self._name = name
        elif not parents and operation is None:
            # Tensor criado diretamente (não resultado de operação)
            Tensor._tensor_counter += 1
            self._name = f"in:{Tensor._tensor_counter}"
        else:
            # Tensor gerado por operação - nome será definido pela operação
            self._name = name if name else "op_result"

    def zero_grad(self):
        """Reinicia o gradiente com zero"""
        if self.requires_grad:
            self.grad = None

    def numpy(self):
        """Retorna o array interno"""
        return self._arr

    def __repr__(self):
        """Permite visualizar os dados do tensor como string"""
        return f"Tensor({self._arr}, name={self._name}, shape={self._arr.shape})"

    def backward(self, my_grad=None):
        """Método usado tanto iniciar o processo de
        diferenciação automática, quanto por um filho
        para enviar o gradiente do pai. No primeiro
        caso, o argumento my_grad não será passado.
        """
        
        # Se requires_grad é False, não fazer nada
        if not self.requires_grad:
            return
        
        # Se my_grad não foi fornecido, este é o ponto de partida da backpropagation
        if my_grad is None:
            # Criar gradiente inicial (vetor de ones com mesma forma que o tensor)
            my_grad = Tensor(np.ones_like(self._arr), requires_grad=False)
        
        # Verificar que o gradiente tem o mesmo shape que o tensor
        assert my_grad.shape == self.shape, f"Gradient shape {my_grad.shape} must match tensor shape {self.shape}"
        
        # SOMAR gradientes (não substituir) - crucial para acumulação
        if self.grad is None:
            self.grad = Tensor(my_grad._arr.copy(), requires_grad=False)
        else:
            # Somar gradientes (para casos onde o mesmo tensor é usado múltiplas vezes)
            assert self.grad.shape == my_grad.shape, f"Existing gradient shape {self.grad.shape} must match new gradient shape {my_grad.shape}"
            self.grad = Tensor(self.grad._arr + my_grad._arr, requires_grad=False)
        
        # Se este tensor tem uma operação que o gerou, calcular gradientes dos pais
        if self._operation is not None and self._parents:
            # A operação calcula os gradientes usando o método grad()
            parent_grads = self._operation.grad(my_grad, *self._parents)
            
            # Verificar que temos um gradiente para cada pai
            assert len(parent_grads) == len(self._parents), f"Number of gradients {len(parent_grads)} must match number of parents {len(self._parents)}"
            
            # Propagar gradientes para os pais
            for parent, parent_grad in zip(self._parents, parent_grads):
                if parent.requires_grad:
                    # Verificar que o gradiente tem o shape correto
                    assert parent_grad.shape == parent.shape, f"Parent gradient shape {parent_grad.shape} must match parent shape {parent.shape}"
                    parent.backward(parent_grad)

    @property
    def shape(self):
        """Retorna a forma do tensor"""
        return self._arr.shape
    
    @property
    def T(self):
        """Retorna a transposta do tensor"""
        return Tensor(self._arr.T, requires_grad=self.requires_grad)

### Interface de  Operações

A classe abaixo define a interface que as operações devem implementar. Ela não precisa ser modificada, mas pode, caso queira.

In [55]:

class Op(ABC):
    @abstractmethod
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando as entradas e
            retorna o tensor resultado. O método deve
            garantir que o atributo parents do tensor
            de saída seja uma lista de tensores."""
        pass

    @abstractmethod
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna os gradientes dos pais em como tensores.

        Arguments:

        - back_grad: Derivada parcial em relação à saída
            da operação backpropagada pelo filho.

        - args: variaveis de entrada da operacao (pais)
            como tensores.

        - O nome dos tensores de gradiente devem ter o
            nome da operacao seguido de '_grad'.
        """
        pass

    @staticmethod
    def _ensure_tensor(value):
        """Converte um valor em Tensor se não for um"""
        if isinstance(value, Tensor):
            return value
        else:
            # Converter escalar, lista, array, etc. em Tensor
            return Tensor(value, requires_grad=False)

    @staticmethod
    # Função para broadcasting
    def _broadcast_tensors(a, b):
        """
        Implementa broadcasting entre dois tensors para compatibilidade de shapes.
        Retorna os tensors com shapes compatíveis para operação element-wise.
        """
        # Se shapes já são iguais, não precisa fazer nada
        if a.shape == b.shape:
            return a, b
        
        # Implementar regras básicas de broadcasting
        shape_a = a.shape
        shape_b = b.shape
        
        # Caso especial: se um é (1,1) e outro é (n,1), expandir o (1,1)
        if shape_a == (1, 1) and shape_b[1] == 1:
            # Expandir 'a' para o shape de 'b'
            broadcasted_arr = np.broadcast_to(a._arr, shape_b)
            new_a = Tensor(broadcasted_arr, requires_grad=a.requires_grad)
            return new_a, b
        elif shape_b == (1, 1) and shape_a[1] == 1:
            # Expandir 'b' para o shape de 'a'
            broadcasted_arr = np.broadcast_to(b._arr, shape_a)
            new_b = Tensor(broadcasted_arr, requires_grad=b.requires_grad)
            return a, new_b
        
        # Caso especial: se um é (1,n) e outro é (m,1), fazer broadcast para (m,n)
        if shape_a[0] == 1 and shape_b[1] == 1:
            target_shape = (shape_b[0], shape_a[1])
            broadcasted_a = np.broadcast_to(a._arr, target_shape)
            broadcasted_b = np.broadcast_to(b._arr, target_shape)
            new_a = Tensor(broadcasted_a, requires_grad=a.requires_grad)
            new_b = Tensor(broadcasted_b, requires_grad=b.requires_grad)
            return new_a, new_b
        elif shape_b[0] == 1 and shape_a[1] == 1:
            target_shape = (shape_a[0], shape_b[1])
            broadcasted_a = np.broadcast_to(a._arr, target_shape)
            broadcasted_b = np.broadcast_to(b._arr, target_shape)
            new_a = Tensor(broadcasted_a, requires_grad=a.requires_grad)
            new_b = Tensor(broadcasted_b, requires_grad=b.requires_grad)
            return new_a, new_b
        
        # Se chegou aqui, os shapes não são compatíveis para broadcasting
        raise ValueError(f"Shapes {shape_a} and {shape_b} are not compatible for broadcasting")

    @staticmethod
    # Função para calcular gradientes com broadcasting
    def _unbroadcast_gradient(grad, original_shape):
        """
        Reduz um gradiente que foi expandido por broadcasting de volta ao shape original.
        """
        if grad.shape == original_shape:
            return grad
        
        grad_arr = grad._arr
        
        # Se o shape original era (1,1), somar todos os elementos
        if original_shape == (1, 1):
            summed = np.sum(grad_arr)
            return Tensor([[summed]], requires_grad=False)
        
        # Se uma dimensão era 1, somar ao longo dessa dimensão
        if original_shape[0] == 1 and grad.shape[0] > 1:
            # Somar ao longo da primeira dimensão
            summed = np.sum(grad_arr, axis=0, keepdims=True)
            return Tensor(summed, requires_grad=False)
        
        if original_shape[1] == 1 and grad.shape[1] > 1:
            # Somar ao longo da segunda dimensão
            summed = np.sum(grad_arr, axis=1, keepdims=True)
            return Tensor(summed, requires_grad=False)
        
        # Se chegou aqui, retornar o gradiente original
        return grad

### Implementação das Operações

Operações devem herdar de `Op` e implementar os métodos `__call__` e `grad`.

Pelo menos as seguintes operações devem ser implementadas:



In [56]:

class Add(Op):
    """Add(a, b): a + b"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""
        assert len(args) == 2, f"Add operation requires exactly 2 arguments, got {len(args)}"
        
        # Converter argumentos para Tensors se necessário
        original_a, original_b = self._ensure_tensor(args[0]), self._ensure_tensor(args[1])
        
        # Fazer broadcasting dos tensors
        a, b = self._broadcast_tensors(original_a, original_b)
        
        # Agora os shapes devem ser iguais
        assert a.shape == b.shape, f"After broadcasting, tensors must have same shape: {a.shape} vs {b.shape}"
        
        # Realizar a operação
        result_arr = a._arr + b._arr
        
        # Verificar que o resultado tem o shape esperado
        assert result_arr.shape == a.shape, f"Result shape {result_arr.shape} should match input shape {a.shape}"
        
        # Criar tensor resultado com referências aos pais ORIGINAIS
        parents = [original_a, original_b]
        
        result = Tensor(
            result_arr,
            parents=parents,
            requires_grad=(original_a.requires_grad or original_b.requires_grad),
            operation=self,
            name=NameManager.new("add")
            
        )
        
        # Guardar informações sobre broadcasting para usar no gradiente
        result._broadcast_info = {
            'original_shapes': (original_a.shape, original_b.shape),
            'broadcasted_shapes': (a.shape, b.shape)
        }
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""
        assert len(args) == 2, f"Add grad requires exactly 2 parent tensors, got {len(args)}"
        
        original_a, original_b = args
        
        # Para soma, o gradiente é o mesmo para ambos os operandos
        grad_a = Tensor(back_grad._arr.copy(), requires_grad=False, name="Add_grad")
        grad_b = Tensor(back_grad._arr.copy(), requires_grad=False, name="Add_grad")
        
        # Fazer unbroadcast dos gradientes para os shapes originais
        grad_a = self._unbroadcast_gradient(grad_a, original_a.shape)
        grad_b = self._unbroadcast_gradient(grad_b, original_b.shape)
        
        # Verificar shapes dos gradientes
        assert grad_a.shape == original_a.shape, f"Gradient shape {grad_a.shape} must match parent shape {original_a.shape}"
        assert grad_b.shape == original_b.shape, f"Gradient shape {grad_b.shape} must match parent shape {original_b.shape}"
        
        return [grad_a, grad_b]

# Instancia a operação
add = Add()

In [57]:

class Sub(Op):
    """Sub(a, b): a - b"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a subtração usando os argumentos dados em args"""
        assert len(args) == 2, f"Sub operation requires exactly 2 arguments, got {len(args)}"
        
        # Converter argumentos para Tensors se necessário
        original_a, original_b = self._ensure_tensor(args[0]), self._ensure_tensor(args[1])
        
        # Fazer broadcasting dos tensors
        a, b = self._broadcast_tensors(original_a, original_b)
        
        # Agora os shapes devem ser iguais
        assert a.shape == b.shape, f"After broadcasting, tensors must have same shape: {a.shape} vs {b.shape}"
        
        # Realizar a operação
        result_arr = a._arr - b._arr
        
        # Verificar que o resultado tem o shape esperado
        assert result_arr.shape == a.shape, f"Result shape {result_arr.shape} should match input shape {a.shape}"
        
        # Criar tensor resultado com referências aos pais ORIGINAIS
        parents = [original_a, original_b]
        
        result = Tensor(
            result_arr,
            parents=parents,
            requires_grad=(original_a.requires_grad or original_b.requires_grad),
            operation=self,
            name=NameManager.new("sub")
        )
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais"""
        assert len(args) == 2, f"Sub grad requires exactly 2 parent tensors, got {len(args)}"
        
        original_a, original_b = args
        
        # Para subtração: d/da (a - b) = 1, d/db (a - b) = -1
        grad_a = Tensor(back_grad._arr.copy(), requires_grad=False, name="Sub_grad")
        grad_b = Tensor(-back_grad._arr.copy(), requires_grad=False, name="Sub_grad")
        
        # Fazer unbroadcast dos gradientes para os shapes originais
        grad_a = self._unbroadcast_gradient(grad_a, original_a.shape)
        grad_b = self._unbroadcast_gradient(grad_b, original_b.shape)
        
        # Verificar shapes dos gradientes
        assert grad_a.shape == original_a.shape, f"Gradient shape {grad_a.shape} must match parent shape {original_a.shape}"
        assert grad_b.shape == original_b.shape, f"Gradient shape {grad_b.shape} must match parent shape {original_b.shape}"
        
        return [grad_a, grad_b]
    
# Instancia a classe. O objeto passa a poder ser usado como uma funcao
sub = Sub()

In [58]:

class Prod(Op):
    """Prod(a, b): produto element-wise a * b (igual ao Mul)"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza o produto element-wise"""
        assert len(args) == 2, f"Prod operation requires exactly 2 arguments, got {len(args)}"
        
        # Converter argumentos para Tensors se necessário
        original_a, original_b = self._ensure_tensor(args[0]), self._ensure_tensor(args[1])
        
        # Fazer broadcasting dos tensors
        a, b = self._broadcast_tensors(original_a, original_b)
        
        # Agora os shapes devem ser iguais
        assert a.shape == b.shape, f"After broadcasting, tensors must have same shape: {a.shape} vs {b.shape}"
        
        # Realizar a operação (produto element-wise)
        result_arr = a._arr * b._arr
        
        # Verificar que o resultado tem o shape esperado
        assert result_arr.shape == a.shape, f"Result shape {result_arr.shape} should match input shape {a.shape}"
        
        # Criar tensor resultado com referências aos pais ORIGINAIS
        parents = [original_a, original_b]
        
        result = Tensor(
            result_arr,
            parents=parents,
            requires_grad=(original_a.requires_grad or original_b.requires_grad),
            operation=self,
            name=NameManager.new("prod")
        )
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais"""
        assert len(args) == 2, f"Prod grad requires exactly 2 parent tensors, got {len(args)}"
        
        original_a, original_b = args
        
        # Fazer broadcasting novamente para calcular gradientes
        a, b = self._broadcast_tensors(original_a, original_b)
        
        # d/da (a * b) = b, d/db (a * b) = a
        grad_a = Tensor(back_grad._arr * b._arr, requires_grad=False, name="Prod_grad")
        grad_b = Tensor(back_grad._arr * a._arr, requires_grad=False, name="Prod_grad")
        
        # Fazer unbroadcast dos gradientes para os shapes originais
        grad_a = self._unbroadcast_gradient(grad_a, original_a.shape)
        grad_b = self._unbroadcast_gradient(grad_b, original_b.shape)
        
        # Verificar shapes dos gradientes
        assert grad_a.shape == original_a.shape, f"Gradient shape {grad_a.shape} must match parent shape {original_a.shape}"
        assert grad_b.shape == original_b.shape, f"Gradient shape {grad_b.shape} must match parent shape {original_b.shape}"
        
        return [grad_a, grad_b]
    
# Instancia a operação
prod = Prod()

In [59]:

class Sin(Op):
    """Sin(a): seno element-wise"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza o seno element-wise"""
        assert len(args) == 1, f"Sin operation requires exactly 1 argument, got {len(args)}"
        
        a = self._ensure_tensor(args[0])
        
        # Calcular seno element-wise
        result_arr = np.sin(a._arr)
        
        # Verificar que o resultado tem o shape esperado
        assert result_arr.shape == a.shape, f"Result shape {result_arr.shape} should match input shape {a.shape}"
        
        # Criar tensor resultado
        result = Tensor(
            result_arr,
            parents=[a],
            requires_grad=a.requires_grad,
            operation=self,
            name=NameManager.new("sin")
        )
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais"""
        assert len(args) == 1, f"Sin grad requires exactly 1 parent tensor, got {len(args)}"
        
        a = args[0]
        
        # Verificar que back_grad tem o shape correto
        assert back_grad.shape == a.shape, f"Back gradient shape {back_grad.shape} must match tensor shape {a.shape}"
        
        # Para seno: d/da sin(a) = cos(a)
        grad_a = Tensor(back_grad._arr * np.cos(a._arr), requires_grad=False, name="Sin_grad")
        
        # Verificar shape do gradiente
        assert grad_a.shape == a.shape, f"Gradient shape {grad_a.shape} must match parent shape {a.shape}"
        
        return [grad_a]
    
# Instancia a classe. O objeto passa a poder ser usado como uma funcao
sin = Sin()

In [60]:

class Cos(Op):
    """Cos(a): cosseno element-wise"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza o cosseno element-wise"""
        assert len(args) == 1, f"Cos operation requires exactly 1 argument, got {len(args)}"
        
        a = self._ensure_tensor(args[0])
        
        # Calcular cosseno element-wise
        result_arr = np.cos(a._arr)
        
        # Verificar que o resultado tem o shape esperado
        assert result_arr.shape == a.shape, f"Result shape {result_arr.shape} should match input shape {a.shape}"
        
        # Criar tensor resultado
        result = Tensor(
            result_arr,
            parents=[a],
            requires_grad=a.requires_grad,
            operation=self,
            name=NameManager.new("cos")
        )
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais"""
        assert len(args) == 1, f"Cos grad requires exactly 1 parent tensor, got {len(args)}"
        
        a = args[0]
        
        # Verificar que back_grad tem o shape correto
        assert back_grad.shape == a.shape, f"Back gradient shape {back_grad.shape} must match tensor shape {a.shape}"
        
        # Para cosseno: d/da cos(a) = -sin(a)
        grad_a = Tensor(back_grad._arr * (-np.sin(a._arr)), requires_grad=False, name="Cos_grad")
        
        # Verificar shape do gradiente
        assert grad_a.shape == a.shape, f"Gradient shape {grad_a.shape} must match parent shape {a.shape}"
        
        return [grad_a]
    
# Instancia a classe. O objeto passa a poder ser usado como uma funcao
cos = Cos()

In [61]:

class Sum(Op):
    """Sum(a): soma todos os elementos do tensor"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a soma de todos os elementos"""
        assert len(args) == 1, f"Sum operation requires exactly 1 argument, got {len(args)}"
        
        a = self._ensure_tensor(args[0])
        
        # Somar todos os elementos
        result_arr = np.array([[np.sum(a._arr)]])
        
        # Criar tensor resultado (sempre shape (1,1))
        result = Tensor(
            result_arr,
            parents=[a],
            requires_grad=a.requires_grad,
            operation=self,
            name=NameManager.new("sum")
        )
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais"""
        assert len(args) == 1, f"Sum grad requires exactly 1 parent tensor, got {len(args)}"
        
        a = args[0]
        
        # Verificar que back_grad é escalar
        assert back_grad.shape == (1, 1), f"Sum back gradient must be scalar, got shape {back_grad.shape}"
        
        # Para soma, o gradiente é o mesmo valor para todos os elementos
        grad_a = Tensor(np.full(a.shape, back_grad._arr[0, 0]), requires_grad=False, name="Sum_grad")
        
        # Verificar shape do gradiente
        assert grad_a.shape == a.shape, f"Gradient shape {grad_a.shape} must match parent shape {a.shape}"
        
        return [grad_a]
    
# Instancia a classe. O objeto passa a poder ser usado como uma funcao
# ⚠️ vamos chamar de my_sum porque python ja possui uma funcao sum
my_sum = Sum()

In [62]:

class Mean(Op):
    """Mean(a): média de todos os elementos do tensor"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a média de todos os elementos"""
        assert len(args) == 1, f"Mean operation requires exactly 1 argument, got {len(args)}"
        
        a = self._ensure_tensor(args[0])
        
        # Calcular média de todos os elementos
        result_arr = np.array([[np.mean(a._arr)]])
        
        # Criar tensor resultado (sempre shape (1,1))
        result = Tensor(
            result_arr,
            parents=[a],
            requires_grad=a.requires_grad,
            operation=self,
            name=NameManager.new("mean")
        )
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais"""
        assert len(args) == 1, f"Mean grad requires exactly 1 parent tensor, got {len(args)}"
        
        a = args[0]
        
        # Verificar que back_grad é escalar
        assert back_grad.shape == (1, 1), f"Mean back gradient must be scalar, got shape {back_grad.shape}"
        
        # Para média, o gradiente é 1/n para cada elemento
        n = a.shape[0] * a.shape[1]  # número total de elementos
        grad_value = back_grad._arr[0, 0] / n
        grad_a = Tensor(np.full(a.shape, grad_value), requires_grad=False, name="Mean_grad")
        
        # Verificar shape do gradiente
        assert grad_a.shape == a.shape, f"Gradient shape {grad_a.shape} must match parent shape {a.shape}"
        
        return [grad_a]
    
# Instancia a classe. O objeto passa a poder ser usado como uma funcao
mean = Mean()


In [63]:

class Square(Op):
    """Square(a): a^2 element-wise"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza o quadrado element-wise"""
        assert len(args) == 1, f"Square operation requires exactly 1 argument, got {len(args)}"
        
        a = self._ensure_tensor(args[0])
        
        # Elevar ao quadrado element-wise
        result_arr = a._arr ** 2
        
        # Verificar que o resultado tem o shape esperado
        assert result_arr.shape == a.shape, f"Result shape {result_arr.shape} should match input shape {a.shape}"
        
        # Criar tensor resultado
        result = Tensor(
            result_arr,
            parents=[a],
            requires_grad=a.requires_grad,
            operation=self,
            name=NameManager.new("square")
        )
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais"""
        assert len(args) == 1, f"Square grad requires exactly 1 parent tensor, got {len(args)}"
        
        a = args[0]
        
        # Verificar que back_grad tem o shape correto
        assert back_grad.shape == a.shape, f"Back gradient shape {back_grad.shape} must match tensor shape {a.shape}"
        
        # Para quadrado: d/da (a^2) = 2a
        grad_a = Tensor(back_grad._arr * 2 * a._arr, requires_grad=False, name="Square_grad")
        
        # Verificar shape do gradiente
        assert grad_a.shape == a.shape, f"Gradient shape {grad_a.shape} must match parent shape {a.shape}"
        
        return [grad_a]
    
# Instancia a classe. O objeto passa a poder ser usado como uma funcao
square = Square()

In [64]:

class MatMul(Op):
    """MatMul(a, b): a @ b (matrix multiplication)"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a multiplicação matricial"""
        assert len(args) == 2, f"MatMul operation requires exactly 2 arguments, got {len(args)}"
        
        # Converter argumentos para Tensors se necessário
        a, b = self._ensure_tensor(args[0]), self._ensure_tensor(args[1])
        
        # Verificar compatibilidade de dimensões para multiplicação matricial
        assert len(a.shape) == 2 and len(b.shape) == 2, f"MatMul requires 2D tensors, got shapes {a.shape} and {b.shape}"
        assert a.shape[1] == b.shape[0], f"Incompatible shapes for matrix multiplication: {a.shape} and {b.shape}"
        
        # Realizar a operação
        result_arr = a._arr @ b._arr
        
        # Verificar que o resultado tem o shape esperado
        expected_shape = (a.shape[0], b.shape[1])
        assert result_arr.shape == expected_shape, f"Result shape {result_arr.shape} should be {expected_shape}"
        
        # Criar tensor resultado
        parents = [a, b]
        
        result = Tensor(
            result_arr,
            parents=parents,
            requires_grad=(a.requires_grad or b.requires_grad),
            operation=self,
            name=NameManager.new("matmul")
        )
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais"""
        assert len(args) == 2, f"MatMul grad requires exactly 2 parent tensors, got {len(args)}"
        
        a, b = args
        
        # Verificar que back_grad tem o shape correto (resultado da multiplicação)
        expected_grad_shape = (a.shape[0], b.shape[1])
        assert back_grad.shape == expected_grad_shape, f"Back gradient shape {back_grad.shape} must match result shape {expected_grad_shape}"
        
        # d/da (a @ b) = back_grad @ b.T
        # d/db (a @ b) = a.T @ back_grad
        grad_a = Tensor(back_grad._arr @ b._arr.T, requires_grad=False, name="matmul_grad")
        grad_b = Tensor(a._arr.T @ back_grad._arr, requires_grad=False, name="matmul_grad")
        
        # Verificar shapes dos gradientes
        assert grad_a.shape == a.shape, f"Gradient a shape {grad_a.shape} must match parent shape {a.shape}"
        assert grad_b.shape == b.shape, f"Gradient b shape {grad_b.shape} must match parent shape {b.shape}"
        
        return [grad_a, grad_b]
    
# Instancia a classe. O objeto passa a poder ser usado como uma funcao
matmul = MatMul()

In [65]:

class Exp(Op):
    """Exp(a): exponencial element-wise"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a exponencial element-wise"""
        assert len(args) == 1, f"Exp operation requires exactly 1 argument, got {len(args)}"
        
        a = self._ensure_tensor(args[0])
        
        # Calcular exponencial element-wise
        result_arr = np.exp(a._arr)
        
        # Verificar que o resultado tem o shape esperado
        assert result_arr.shape == a.shape, f"Result shape {result_arr.shape} should match input shape {a.shape}"
        
        # Criar tensor resultado
        result = Tensor(
            result_arr,
            parents=[a],
            requires_grad=a.requires_grad,
            operation=self,
            name=NameManager.new("exp")
        )
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais"""
        assert len(args) == 1, f"Exp grad requires exactly 1 parent tensor, got {len(args)}"
        
        a = args[0]
        
        # Verificar que back_grad tem o shape correto
        assert back_grad.shape == a.shape, f"Back gradient shape {back_grad.shape} must match tensor shape {a.shape}"
        
        # Para exponencial: d/da exp(a) = exp(a)
        grad_a = Tensor(back_grad._arr * np.exp(a._arr), requires_grad=False, name="Exp_grad")
        
        # Verificar shape do gradiente
        assert grad_a.shape == a.shape, f"Gradient shape {grad_a.shape} must match parent shape {a.shape}"
        
        return [grad_a]
    
# Instancia a classe. O objeto passa a poder ser usado como uma funcao
exp = Exp()

In [66]:

class ReLU(Op):
    """Relu(a): ReLU element-wise"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza ReLU element-wise"""
        assert len(args) == 1, f"Relu operation requires exactly 1 argument, got {len(args)}"
        
        a = self._ensure_tensor(args[0])
        
        # Calcular ReLU element-wise: max(0, a)
        result_arr = np.maximum(0.0, a._arr)
        
        # Verificar que o resultado tem o shape esperado
        assert result_arr.shape == a.shape, f"Result shape {result_arr.shape} should match input shape {a.shape}"
        
        # Criar tensor resultado
        result = Tensor(
            result_arr,
            parents=[a],
            requires_grad=a.requires_grad,
            operation=self,
            name=NameManager.new("relu")
        )
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais"""
        assert len(args) == 1, f"Relu grad requires exactly 1 parent tensor, got {len(args)}"
        
        a = args[0]
        
        # Verificar que back_grad tem o shape correto
        assert back_grad.shape == a.shape, f"Back gradient shape {back_grad.shape} must match tensor shape {a.shape}"
        
        # Para ReLU: d/da ReLU(a) = 1 se a > 0, senão 0
        relu_grad = np.where(a._arr > 0, 1.0, 0.0)
        grad_a = Tensor(back_grad._arr * relu_grad, requires_grad=False, name="Relu_grad")
        
        # Verificar shape do gradiente
        assert grad_a.shape == a.shape, f"Gradient shape {grad_a.shape} must match parent shape {a.shape}"
        
        return [grad_a]
    
# Instancia a classe. O objeto passa a poder ser usado como uma funcao
relu = ReLU()

In [67]:

class Sigmoid(Op):
    """Sigmoid(a): função sigmoide element-wise"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza sigmoide element-wise"""
        assert len(args) == 1, f"Sigmoid operation requires exactly 1 argument, got {len(args)}"
        
        a = self._ensure_tensor(args[0])
        
        # Calcular sigmoide element-wise: 1 / (1 + exp(-a))
        # Usar clip para evitar overflow
        clipped = np.clip(a._arr, -500, 500)
        result_arr = 1.0 / (1.0 + np.exp(-clipped))
        
        # Verificar que o resultado tem o shape esperado
        assert result_arr.shape == a.shape, f"Result shape {result_arr.shape} should match input shape {a.shape}"
        
        # Criar tensor resultado
        result = Tensor(
            result_arr,
            parents=[a],
            requires_grad=a.requires_grad,
            operation=self,
            name=NameManager.new("sigmoid")
        )
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais"""
        assert len(args) == 1, f"Sigmoid grad requires exactly 1 parent tensor, got {len(args)}"
        
        a = args[0]
        
        # Verificar que back_grad tem o shape correto
        assert back_grad.shape == a.shape, f"Back gradient shape {back_grad.shape} must match tensor shape {a.shape}"
        
        # Para sigmoide: d/da sigmoid(a) = sigmoid(a) * (1 - sigmoid(a))
        clipped = np.clip(a._arr, -500, 500)
        sigmoid_val = 1.0 / (1.0 + np.exp(-clipped))
        sigmoid_grad = sigmoid_val * (1.0 - sigmoid_val)
        grad_a = Tensor(back_grad._arr * sigmoid_grad, requires_grad=False, name="Sigmoid_grad")
        
        # Verificar shape do gradiente
        assert grad_a.shape == a.shape, f"Gradient shape {grad_a.shape} must match parent shape {a.shape}"
        
        return [grad_a]

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
sigmoid = Sigmoid()

In [68]:

class Tanh(Op):
    """Tanh(a): tangente hiperbólica element-wise"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza tanh element-wise"""
        assert len(args) == 1, f"Tanh operation requires exactly 1 argument, got {len(args)}"
        
        a = self._ensure_tensor(args[0])
        
        # Calcular tanh element-wise
        result_arr = np.tanh(a._arr)
        
        # Verificar que o resultado tem o shape esperado
        assert result_arr.shape == a.shape, f"Result shape {result_arr.shape} should match input shape {a.shape}"
        
        # Criar tensor resultado
        result = Tensor(
            result_arr,
            parents=[a],
            requires_grad=a.requires_grad,
            operation=self,
            name=NameManager.new("tanh")
        )
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais"""
        assert len(args) == 1, f"Tanh grad requires exactly 1 parent tensor, got {len(args)}"
        
        a = args[0]
        
        # Verificar que back_grad tem o shape correto
        assert back_grad.shape == a.shape, f"Back gradient shape {back_grad.shape} must match tensor shape {a.shape}"
        
        # Para tanh: d/da tanh(a) = 1 - tanh²(a)
        tanh_val = np.tanh(a._arr)
        tanh_grad = 1.0 - tanh_val**2
        grad_a = Tensor(back_grad._arr * tanh_grad, requires_grad=False, name="Tanh_grad")
        
        # Verificar shape do gradiente
        assert grad_a.shape == a.shape, f"Gradient shape {grad_a.shape} must match parent shape {a.shape}"
        
        return [grad_a]

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
tanh = Tanh()

In [69]:

class Softmax(Op):
    """Softmax(a): função softmax aplicada na dimensão apropriada"""
    
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza softmax na dimensão apropriada"""
        assert len(args) == 1, f"Softmax operation requires exactly 1 argument, got {len(args)}"
        
        a = self._ensure_tensor(args[0])
        
        # Determinar o eixo correto para softmax
        # Se é um vetor coluna (n, 1), aplicar softmax ao longo do eixo 0
        # Se é um vetor linha (1, n), aplicar softmax ao longo do eixo 1
        # Se é uma matriz (m, n) com m > 1 e n > 1, aplicar ao longo do eixo 1 (features)
        
        if a.shape[1] == 1:  # Vetor coluna (n, 1)
            axis = 0
        else:  # Matriz geral ou vetor linha, aplicar ao longo do eixo 1
            axis = 1
        
        # Calcular softmax ao longo do eixo determinado
        # Subtrair o máximo para estabilidade numérica
        max_vals = np.max(a._arr, axis=axis, keepdims=True)
        exp_vals = np.exp(a._arr - max_vals)
        sum_exp = np.sum(exp_vals, axis=axis, keepdims=True)
        result_arr = exp_vals / sum_exp
        
        # Verificar que o resultado tem o shape esperado
        assert result_arr.shape == a.shape, f"Result shape {result_arr.shape} should match input shape {a.shape}"
        
        # Criar tensor resultado
        result = Tensor(
            result_arr,
            parents=[a],
            requires_grad=a.requires_grad,
            operation=self,
            name=NameManager.new("softmax")
        )
        
        # Guardar informações para o gradiente
        result._softmax_axis = axis
        
        return result
    
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais"""
        assert len(args) == 1, f"Softmax grad requires exactly 1 parent tensor, got {len(args)}"
        
        a = args[0]
        
        # Verificar que back_grad tem o shape correto
        assert back_grad.shape == a.shape, f"Back gradient shape {back_grad.shape} must match tensor shape {a.shape}"
        
        # Determinar o eixo (mesmo critério do forward)
        if a.shape[1] == 1:  # Vetor coluna (n, 1)
            axis = 0
        else:  # Matriz geral ou vetor linha
            axis = 1
        
        # Calcular softmax novamente para obter os valores
        max_vals = np.max(a._arr, axis=axis, keepdims=True)
        exp_vals = np.exp(a._arr - max_vals)
        sum_exp = np.sum(exp_vals, axis=axis, keepdims=True)
        softmax_vals = exp_vals / sum_exp
        
        # Para softmax: jacobiano é softmax[i] * (δ[i,j] - softmax[j])
        # Implementação eficiente: s * (back_grad - (s * back_grad).sum(axis, keepdims=True))
        grad_arr = softmax_vals * (back_grad._arr - np.sum(softmax_vals * back_grad._arr, axis=axis, keepdims=True))
        grad_a = Tensor(grad_arr, requires_grad=False, name="Softmax_grad")
        
        # Verificar shape do gradiente
        assert grad_a.shape == a.shape, f"Gradient shape {grad_a.shape} must match parent shape {a.shape}"
        
        return [grad_a]
    
# Instancia a classe. O objeto passa a poder ser usado como uma funcao
softmax = Softmax()


### ‼️ Regras e Pontos de Atenção‼️

- Vamos fazer a hipótese simplificadora que Tensores devem ser sempre matrizes. Por exemplo, o escalar 2 deve ser armazado em `_arr` como a matriz `[[2]]`. De forma similar, a lista `[1, 2, 3]` deve ser armazenada em `_arr` como em uma matriz coluna.

- Devem ser realizados `asserts` nas operações para garantir que os shapes dos operandos fazem sentido. Esta verificação também deve ser feita depois das operações que manipulam gradientes de tensores.

- Devem ser respeitados os nomes dos atributos, métodos e classes para viabilizar os testes automáticos.

- Gradientes devem ser calculados usando uma passada pelo grafo computacional.

- Os gradientes devem ser somados e não substituídos nas chamadas de  backward. Isto vai permitir que os gradientes sejam acumulados entre amostras do dataset e que os resultados sejam corretos mesmo em caso de ramificações e junções no grafo computacional.

- Lembre-se de zerar os gradientes após cada passo de gradient descent (atualização dos parâmetros).


## Testes Básicos

Estes testes avaliam se a derivada da função está sendo calculada corretamente, mas em muitos casos **não** avaliam se os gradientes backpropagados estão sendo incorporados corretamente. Esta avaliação será feita nos problemas da próxima seção.

Operador de Soma

In [70]:
# add

a = Tensor([1.0, 2.0, 3.0])
b = Tensor([4.0, 5.0, 6.0])
c = add(a, b)
d = add(c, 3.0)
d.backward()

# esperado: matrizes coluna contendo 1
print(a.grad)
print(b.grad)


Tensor([[1.]
 [1.]
 [1.]], name=in:9, shape=(3, 1))
Tensor([[1.]
 [1.]
 [1.]], name=in:10, shape=(3, 1))


Operador de Subtração

In [71]:
# sub

a = Tensor([1.0, 2.0, 3.0])
b = Tensor([4.0, 5.0, 6.0])
c = sub(a, b)
d = sub(c, 3.0)
d.backward()

# esperado: matrizes coluna contendo 1 e -1
print(a.grad)
print(b.grad)


Tensor([[1.]
 [1.]
 [1.]], name=in:19, shape=(3, 1))
Tensor([[-1.]
 [-1.]
 [-1.]], name=in:20, shape=(3, 1))


Operador de Produto

In [72]:
# prod

a = Tensor([1.0, 2.0, 3.0])
b = Tensor([4.0, 5.0, 6.0])
c = prod(a, b)
d = prod(c, 3.0)
d.backward()

# esperado: [12, 15, 18]^T
print(a.grad)
# esperado: [3, 6, 9]^T
print(b.grad)


Tensor([[12.]
 [15.]
 [18.]], name=in:30, shape=(3, 1))
Tensor([[3.]
 [6.]
 [9.]], name=in:31, shape=(3, 1))


Operadores trigonométricos

In [73]:
# sin e cos

a = Tensor([np.pi, 0, np.pi/2])
b = sin(a)
c = cos(a)
d = my_sum(add(b, c))
d.backward()

# esperado: [-1, 1, -1]^T
print(a.grad)

Tensor([[-1.]
 [ 1.]
 [-1.]], name=in:39, shape=(3, 1))


In [74]:
# Sum

a = Tensor([3.0, 1.0, 0.0, 2.0])
b = add(prod(a, 3.0), a)
c = my_sum(b)
c.backward()

# esperado: [4, 4, 4, 4]^T
print(a.grad)

print("\n\na.grad =", a.grad)


Tensor([[4.]
 [4.]
 [4.]
 [4.]], name=in:50, shape=(4, 1))


a.grad = Tensor([[4.]
 [4.]
 [4.]
 [4.]], name=in:50, shape=(4, 1))


In [75]:
# Mean

a = Tensor([3.0, 1.0, 0.0, 2.0])
b = mean(a)
b.backward()

# esperado: [0.25, 0.25, 0.25, 0.25]^T
print(a.grad)


Tensor([[0.25]
 [0.25]
 [0.25]
 [0.25]], name=in:54, shape=(4, 1))


In [76]:
# Square

a = Tensor([3.0, 1.0, 0.0, 2.0])
b = square(a)

# esperado: [9, 1, 0, 4]^T
print(b)

b.backward()

# esperado: [6, 2, 0, 4]
print(a.grad)

Tensor([[9.]
 [1.]
 [0.]
 [4.]], name=square:0, shape=(4, 1))
Tensor([[6.]
 [2.]
 [0.]
 [4.]], name=in:58, shape=(4, 1))


In [77]:
# matmul

W = Tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]
])

v = Tensor([1.0, 2.0, 3.0])

z = matmul(W, v)

# esperado: [14, 32, 50]^T
print(z)

z.backward()

# esperado:
# [1, 2, 3]
# [1, 2, 3]
# [1, 2, 3]
print(W.grad)

# esperado: [12, 15, 18]^T
print(v.grad)


Tensor([[14.]
 [32.]
 [50.]], name=matmul:0, shape=(3, 1))
Tensor([[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]], name=in:63, shape=(3, 3))
Tensor([[12.]
 [15.]
 [18.]], name=in:64, shape=(3, 1))


In [78]:
# Exp

v = Tensor([1.0, 2.0, 3.0])
w = exp(v)

# esperado: [2.718..., 7.389..., 20.085...]^T
print(w)

w.backward()

# esperado: [2.718..., 7.389..., 20.085...]^T
print(v.grad)

Tensor([[ 2.71828183]
 [ 7.3890561 ]
 [20.08553692]], name=exp:0, shape=(3, 1))
Tensor([[ 2.71828183]
 [ 7.3890561 ]
 [20.08553692]], name=in:68, shape=(3, 1))


In [79]:
# Relu

v = Tensor([-1.0, 0.0, 1.0, 3.0])
w = relu(v)

# esperado: [0, 0, 1, 3]^T
print(w)

w.backward()

# esperado: [0, 0, 1, 1]^T
print(v.grad)

Tensor([[0.]
 [0.]
 [1.]
 [3.]], name=relu:0, shape=(4, 1))
Tensor([[0.]
 [0.]
 [1.]
 [1.]], name=in:72, shape=(4, 1))


In [80]:
# Sigmoid

v = Tensor([-1.0, 0.0, 1.0, 3.0])
w = sigmoid(v)

# esperado: [0.268.., 0.5, 0.731.., 0.952..]^T
print(w)

w.backward()

# esperado: [0.196..., 0.25, 0.196..., 0.045...]^T
print(v.grad)

Tensor([[0.26894142]
 [0.5       ]
 [0.73105858]
 [0.95257413]], name=sigmoid:0, shape=(4, 1))
Tensor([[0.19661193]
 [0.25      ]
 [0.19661193]
 [0.04517666]], name=in:76, shape=(4, 1))


In [81]:
# Tanh

v = Tensor([-1.0, 0.0, 1.0, 3.0])
w = tanh(v)

# esperado: [[-0.76159416, 0., 0.76159416, 0.99505475]^T
print(w)

w.backward()

# esperado: [0.41997434, 1., 0.41997434, 0.00986604]^T
print(v.grad)

Tensor([[-0.76159416]
 [ 0.        ]
 [ 0.76159416]
 [ 0.99505475]], name=tanh:0, shape=(4, 1))
Tensor([[0.41997434]
 [1.        ]
 [0.41997434]
 [0.00986604]], name=in:80, shape=(4, 1))


In [82]:
# Softmax

x = Tensor([-3.1, 0.5, 1.0, 2.0])
y = softmax(x)

# esperado: [0.00381737, 0.13970902, 0.23034123, 0.62613238]^T
print(y)

# como exemplo, calcula o MSE para um target vector
diff = sub(y, [1, 0, 0, 0])
sq = square(diff)
a = mean(sq)

# esperado: 0.36424932
print("MSE:", a)

a.backward()

# esperado: [-0.00278095, -0.02243068, -0.02654377, 0.05175539]^T
print(x.grad)



Tensor([[0.00381737]
 [0.13970902]
 [0.23034123]
 [0.62613238]], name=softmax:0, shape=(4, 1))
MSE: Tensor([[0.36424932]], name=mean:1, shape=(1, 1))
Tensor([[-0.00278095]
 [-0.02243068]
 [-0.02654377]
 [ 0.05175539]], name=in:88, shape=(4, 1))


## Referências

### Principais

- [Build your own pytorch](https://www.peterholderrieth.com/blog/2023/Build-Your-Own-Pytorch-1-Computation-Graphs/)
- [Build your own Pytorch - 2: Backpropagation](https://www.peterholderrieth.com/blog/2023/Build-Your-Own-Pytorch-2-Autograd/)
- [Build your own PyTorch - 3: Training a Neural Network with self-made AD software](https://www.peterholderrieth.com/blog/2023/Build-Your-Own-Pytorch-3-Build-Classifier/)
- [Pytorch: A Gentle Introduction to torch.autograd](https://docs.pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html)
- [Automatic Differentiation with torch.autograd](https://docs.pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html)

### Secundárias

- [Tom Roth: Building a computational graph: part 1](https://tomroth.dev/compgraph1/)
- [Tom Roth: Building a computational graph: part 2](https://tomroth.dev/compgraph2/)
- [Tom Roth: Building a computational graph: part 3](https://tomroth.dev/compgraph3/)
- [Roger Grosse (Toronto) class on Automatic Differentiation](https://www.cs.toronto.edu/~rgrosse/courses/csc321_2018/slides/lec10.pdf)
- [Computational graphs and gradient flows](https://simple-english-machine-learning.readthedocs.io/en/latest/neural-networks/computational-graphs.html)
- [Colah Visual Blog: Backprop](https://colah.github.io/posts/2015-08-Backprop/)
- [Towards Data Science: Automatic Differentiation (AutoDiff): A Brief Intro with Examples](https://towardsdatascience.com/automatic-differentiation-autodiff-a-brief-intro-with-examples-3f3d257ffe3b/)
- [A Hands-on Introduction to Automatic Differentiation - Part 1](https://mostafa-samir.github.io/auto-diff-pt1/)
- [Build Your own Deep Learning Framework - A Hands-on Introduction to Automatic Differentiation - Part 2](https://mostafa-samir.github.io/auto-diff-pt1/)
